In [ ]:
seed = 43 # (42, 43, 111)

In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
text_data = pd.read_csv("../Data_Files/Text.csv")
text_data

In [ ]:
text_data.columns

In [ ]:
from tqdm import tqdm
def label_prep(merged_df):
    vads = pd.read_csv("/content/Text - Text.csv") #getting label csv
    print("reading vad")

    #adding arousal, valence and dominance columns to orginal data csv
    for index, row in tqdm(merged_df.iterrows()):
        matching_rows = vads[(vads['Participant_ID'] == row['Participant_ID']) & (vads['Video_ID'] == row['Video_ID'])]

        if not matching_rows.empty:

            merged_df.at[index, 'Valence'] = matching_rows['Valence'].iloc[0]
            merged_df.at[index, 'Arousal'] = matching_rows['Arousal'].iloc[0]
            # merged_df.at[index, 'Dominance'] = matching_rows['Dominance'].iloc[0]
            # merged_df.at[index, 'significance'] = matching_rows['significance'].iloc[0]

    print("binning...")
    # Define the bins and labels for categorization
    bins = [1, 3, 5]  # Define the bin edges
    labels = [0, 1]   # Define the corresponding labels (0 (Low) for 1-3:, 1 (High) for 4-5)

    # Use the cut function to categorize the 'arousal' column
    merged_df['arousal_category'] = pd.cut(merged_df['Arousal'], bins=bins, labels=labels, include_lowest=True)
    merged_df['valence_category'] = pd.cut(merged_df['Valence'], bins=bins, labels=labels, include_lowest=True)

    # Convert the 'category' column to integer type if needed
    merged_df['arousal_category'] = merged_df['arousal_category'].astype(int)
    merged_df['valence_category'] = merged_df['valence_category'].astype(int)

    print("mapping")
    mapping = {
    'Baseline': 0,
    'LVLA': 0,
    'LVHA': 0,
    'HVHA': 1,
    'HVLA': 1  # Baseline and HVLA mapped to 0
    }

    # Apply the mapping to the 'CMA' column
    merged_df['taskwiselabel'] = merged_df['CMA'].map(mapping)
    # autofet_df

    # three_class_mapping = {
    # 'Baseline': 1,
    # 'LVLA': 1,
    # 'LVHA': 0,
    # 'HVHA': 1,
    # 'HVLA': 2
    # }

    # merged_df['three_class_label'] = merged_df['CMA'].map(three_class_mapping)
    return merged_df

text_data = label_prep(text_data)
text_data.to_csv("text_data_labels.csv")


In [ ]:
text_data = pd.read_csv("text_data_labels.csv")
text_data

Pre-Processing and cleaning

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

def lowercase(data):
  temp  = data.lower()
  return temp

def stop_words(text):
  stop_words_set = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words_set]
  return " ".join(filtered_sentence)

def punctuations(data):
  no_punct=[words for words in data if words not in string.punctuation]
  words_wo_punct=''.join(no_punct)
  return words_wo_punct

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  word_tokens = word_tokenize(text)
  lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
  return " ".join(lemmatized_text)

def preprcosess(text_data):
    # text_data['Text'] = text_data['Text Description'].apply(lambda x: lemmatize(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: stop_words(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: lowercase(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: punctuations(x))
    return text_data

text_data = preprcosess(text_data)
text_data

Wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
mCorpus=[]
for row in text_data['Text']:
      mCorpus.extend(row.split(" "))

wordfreq = {}
for word in mCorpus:
    if word not in wordfreq:
        wordfreq[word] = 0
    wordfreq[word] += 1

plt.figure(figsize=(10,10))
wordcloud = WordCloud(background_color="black",
                      width=512,
                      height=384,
                     ).generate(" ".join(wordfreq))

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

Model

Valence Distilbert

In [ ]:
import numpy as np
from datasets import load_metric, Dataset, DatasetDict
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Load your dataset here
# For example, assuming text_valence is a pandas DataFrame with 'Text' and 'valence_category' columns
# vdataset = Dataset.from_pandas(text_valence)

# Example placeholder for the dataset, replace with your actual dataset
from datasets import Dataset
import pandas as pd

text_valence = text_data[['Text','valence_category']]
vdataset = Dataset.from_pandas(text_valence)

# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy['accuracy'], "f1score": f1['f1']}

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare training arguments
batch_size = 16
number_of_epochs = 7

all_metrics = []

for train_index, test_index in kf.split(text_valence):
    train_df = text_valence.iloc[train_index]
    test_df = text_valence.iloc[test_index]

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenize datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    tokenized_train_dataset = tokenized_train_dataset.rename_column("valence_category", "labels")
    tokenized_test_dataset = tokenized_test_dataset.rename_column("valence_category", "labels")

    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

    logging_steps = len(tokenized_train_dataset) // batch_size
    steps = (len(tokenized_train_dataset) / batch_size) * number_of_epochs
    warmup_steps = int(0.2 * steps)

    training_args = TrainingArguments(
        num_train_epochs=number_of_epochs,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        save_strategy='steps',
        learning_rate=2e-5,
        logging_steps=logging_steps,
        warmup_steps=warmup_steps,
        save_steps=1000,
        eval_steps=500,
        output_dir="fine-tuned-distilbert-base-uncased",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    all_metrics.append(eval_results)

# Calculate average metrics
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average metrics over 5 folds:", avg_metrics)

Valence XLM-Bert

In [ ]:
import numpy as np
from datasets import load_metric, Dataset, DatasetDict
from transformers import XLMRobertaTokenizer, RobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Initialize the tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

# Load your dataset here
# For example, assuming text_valence is a pandas DataFrame with 'Text' and 'valence_category' columns
# vdataset = Dataset.from_pandas(text_valence)

# Example placeholder for the dataset, replace with your actual dataset
from datasets import Dataset
import pandas as pd

text_valence = text_data[['Text','valence_category']]
vdataset = Dataset.from_pandas(text_valence)

# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy['accuracy'], "f1score": f1['f1']}

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare training arguments
batch_size = 16
number_of_epochs = 7

all_metrics = []

for train_index, test_index in kf.split(text_valence):
    train_df = text_valence.iloc[train_index]
    test_df = text_valence.iloc[test_index]

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenize datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    tokenized_train_dataset = tokenized_train_dataset.rename_column("valence_category", "labels")
    tokenized_test_dataset = tokenized_test_dataset.rename_column("valence_category", "labels")

    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

    logging_steps = len(tokenized_train_dataset) // batch_size
    steps = (len(tokenized_train_dataset) / batch_size) * number_of_epochs
    warmup_steps = int(0.2 * steps)

    training_args = TrainingArguments(
        num_train_epochs=number_of_epochs,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        save_strategy='steps',
        learning_rate=2e-5,
        logging_steps=logging_steps,
        warmup_steps=warmup_steps,
        save_steps=1000,
        eval_steps=500,
        output_dir="fine-tuned-xlm-base-uncased",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    all_metrics.append(eval_results)

# Calculate average metrics
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average metrics over 5 folds:", avg_metrics)


Arousal Distilbert

In [ ]:
import numpy as np
from datasets import load_metric, Dataset, DatasetDict
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Load your dataset here
# For example, assuming text_valence is a pandas DataFrame with 'Text' and 'valence_category' columns
# vdataset = Dataset.from_pandas(text_valence)

# Example placeholder for the dataset, replace with your actual dataset
from datasets import Dataset
import pandas as pd

text_arousal = text_data[['Text','arousal_category']]
vdataset = Dataset.from_pandas(text_arousal)

# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy['accuracy'], "f1score": f1['f1']}

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare training arguments
batch_size = 16
number_of_epochs = 7

all_metrics = []

for train_index, test_index in kf.split(text_arousal):
    train_df = text_arousal.iloc[train_index]
    test_df = text_arousal.iloc[test_index]

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenize datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    tokenized_train_dataset = tokenized_train_dataset.rename_column("arousal_category", "labels")
    tokenized_test_dataset = tokenized_test_dataset.rename_column("arousal_category", "labels")

    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

    logging_steps = len(tokenized_train_dataset) // batch_size
    steps = (len(tokenized_train_dataset) / batch_size) * number_of_epochs
    warmup_steps = int(0.2 * steps)

    training_args = TrainingArguments(
        num_train_epochs=number_of_epochs,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        save_strategy='steps',
        learning_rate=2e-5,
        logging_steps=logging_steps,
        warmup_steps=warmup_steps,
        save_steps=1000,
        eval_steps=500,
        output_dir="fine-tuned-distilbert-base-uncased",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    all_metrics.append(eval_results)

# Calculate average metrics
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average metrics over 5 folds:", avg_metrics)


Arousal XLMBert

In [ ]:
import numpy as np
from datasets import load_metric, Dataset, DatasetDict
from transformers import XLMRobertaTokenizer, RobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Initialize the tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

# Load your dataset here
# For example, assuming text_valence is a pandas DataFrame with 'Text' and 'valence_category' columns
# vdataset = Dataset.from_pandas(text_valence)

# Example placeholder for the dataset, replace with your actual dataset
from datasets import Dataset
import pandas as pd

text_arousal = text_data[['Text','arousal_category']]
vdataset = Dataset.from_pandas(text_arousal)

# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy['accuracy'], "f1score": f1['f1']}

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare training arguments
batch_size = 16
number_of_epochs = 7

all_metrics = []

for train_index, test_index in kf.split(text_arousal):
    train_df = text_arousal.iloc[train_index]
    test_df = text_arousal.iloc[test_index]

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenize datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    tokenized_train_dataset = tokenized_train_dataset.rename_column("arousal_category", "labels")
    tokenized_test_dataset = tokenized_test_dataset.rename_column("arousal_category", "labels")

    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

    logging_steps = len(tokenized_train_dataset) // batch_size
    steps = (len(tokenized_train_dataset) / batch_size) * number_of_epochs
    warmup_steps = int(0.2 * steps)

    training_args = TrainingArguments(
        num_train_epochs=number_of_epochs,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        save_strategy='steps',
        learning_rate=2e-5,
        logging_steps=logging_steps,
        warmup_steps=warmup_steps,
        save_steps=1000,
        eval_steps=500,
        output_dir="fine-tuned-xlm-base-uncased",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    all_metrics.append(eval_results)

# Calculate average metrics
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average metrics over 5 folds:", avg_metrics)


Taskwise Distilbert

In [ ]:
import numpy as np
from datasets import load_metric, Dataset, DatasetDict
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Load your dataset here
# For example, assuming text_valence is a pandas DataFrame with 'Text' and 'arousal_category' columns
# vdataset = Dataset.from_pandas(text_valence)

# Example placeholder for the dataset, replace with your actual dataset
from datasets import Dataset
import pandas as pd

text_taskwiselabel = text_data[['Text','taskwiselabel']]
vdataset = Dataset.from_pandas(text_taskwiselabel)

# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy['accuracy'], "f1score": f1['f1']}

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare training arguments
batch_size = 16
number_of_epochs = 7

all_metrics = []

for train_index, test_index in kf.split(text_taskwiselabel):
    train_df = text_taskwiselabel.iloc[train_index]
    test_df = text_taskwiselabel.iloc[test_index]

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenize datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    tokenized_train_dataset = tokenized_train_dataset.rename_column("taskwiselabel", "labels")
    tokenized_test_dataset = tokenized_test_dataset.rename_column("taskwiselabel", "labels")

    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

    logging_steps = len(tokenized_train_dataset) // batch_size
    steps = (len(tokenized_train_dataset) / batch_size) * number_of_epochs
    warmup_steps = int(0.2 * steps)

    training_args = TrainingArguments(
        num_train_epochs=number_of_epochs,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        save_strategy='steps',
        learning_rate=2e-5,
        logging_steps=logging_steps,
        warmup_steps=warmup_steps,
        save_steps=1000,
        eval_steps=500,
        output_dir="fine-tuned-distilbert-base-uncased",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    all_metrics.append(eval_results)

# Calculate average metrics
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average metrics over 5 folds:", avg_metrics)


Taskwise XLMBert

In [ ]:
import numpy as np
from datasets import load_metric, Dataset, DatasetDict
from transformers import XLMRobertaTokenizer, RobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold

# Initialize the tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

# Load your dataset here
# For example, assuming text_valence is a pandas DataFrame with 'Text' and 'valence_category' columns
# vdataset = Dataset.from_pandas(text_valence)

# Example placeholder for the dataset, replace with your actual dataset
from datasets import Dataset
import pandas as pd

text_taskwiselabel = text_data[['Text','taskwiselabel']]
vdataset = Dataset.from_pandas(text_taskwiselabel)

# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy['accuracy'], "f1score": f1['f1']}

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare training arguments
batch_size = 16
number_of_epochs = 7

all_metrics = []

for train_index, test_index in kf.split(text_taskwiselabel):
    train_df = text_taskwiselabel.iloc[train_index]
    test_df = text_taskwiselabel.iloc[test_index]

    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Tokenize datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    tokenized_train_dataset = tokenized_train_dataset.rename_column("taskwiselabel", "labels")
    tokenized_test_dataset = tokenized_test_dataset.rename_column("taskwiselabel", "labels")

    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

    logging_steps = len(tokenized_train_dataset) // batch_size
    steps = (len(tokenized_train_dataset) / batch_size) * number_of_epochs
    warmup_steps = int(0.2 * steps)

    training_args = TrainingArguments(
        num_train_epochs=number_of_epochs,
        load_best_model_at_end=True,
        evaluation_strategy='steps',
        save_strategy='steps',
        learning_rate=2e-5,
        logging_steps=logging_steps,
        warmup_steps=warmup_steps,
        save_steps=1000,
        eval_steps=500,
        output_dir="fine-tuned-xlm-base-uncased",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    all_metrics.append(eval_results)

# Calculate average metrics
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average metrics over 5 folds:", avg_metrics)
